### Convert label-studio annotations to YOLO format

In [1]:
from pathlib import Path

In [3]:
HOME = Path.cwd().parent.parent
HOME

PosixPath('/home/ubuntu_wsl/computer_vision/analog_watch_reader')

In [14]:
import json

In [6]:
annotation_file_name = "kp_annotations.json"

In [9]:
KP_ANNOTATIONS = (
    HOME/f"annotations/keypoints/labelstudio/{annotation_file_name}"
    )

with open(KP_ANNOTATIONS) as f:
    data = json.load(f)

annotation1 = data[0]

annotation1

{'id': 43570,
 'annotations': [{'id': 1451,
   'completed_by': 1,
   'result': [{'original_width': 640,
     'original_height': 600,
     'image_rotation': 0,
     'value': {'x': 30.65131128234742,
      'y': 49.49708822395115,
      'width': 0.33333333333333337,
      'keypointlabels': ['12_oclock']},
     'parentID': 'xlP0_7HXY-',
     'id': 'aQ9ZDlTdlf',
     'from_name': 'keypoints',
     'to_name': 'image',
     'type': 'keypointlabels',
     'origin': 'manual'},
    {'original_width': 640,
     'original_height': 600,
     'image_rotation': 0,
     'value': {'x': 47.93362144082724,
      'y': 50.15940430188092,
      'width': 0.33333333333333337,
      'keypointlabels': ['clock_center']},
     'parentID': 'xlP0_7HXY-',
     'id': 'RTUU_Pmxc7',
     'from_name': 'keypoints',
     'to_name': 'image',
     'type': 'keypointlabels',
     'origin': 'manual'},
    {'original_width': 640,
     'original_height': 600,
     'image_rotation': 0,
     'value': {'x': 50.31381984588733,
     

In [10]:
len(data)

1200

In [11]:
annotation1_img_path = data[0]["data"]["image"]
annotation1_img_path

'/data/local-files/?d=images/original/0008c01db70bc73e9bdef2514b75c6a2_jpeg.rf.1db421356484dbaa6774c0112eb35ca9.jpg'

In [12]:
Path(annotation1_img_path).name

'0008c01db70bc73e9bdef2514b75c6a2_jpeg.rf.1db421356484dbaa6774c0112eb35ca9.jpg'

In [13]:
Path(annotation1_img_path).stem

'0008c01db70bc73e9bdef2514b75c6a2_jpeg.rf.1db421356484dbaa6774c0112eb35ca9'

In [18]:
for an in data[0]["annotations"][0]["result"]:
    print(an["value"].get("keypointlabels", "BBox"))

['12_oclock']
['clock_center']
['hour_hand']
['minute_hand']
BBox
['3_oclock']
['6_oclock']
['9_oclock']


In [21]:
from typing import Literal

In [22]:
def convert_ls_to_yolo_kpt(ls_json_path:Path, 
                        output_dir:Path, 
                        class_map:dict, 
                        dims:Literal[2,3]):
    # Ensure output directory exists
    # os.makedirs(output_dir, exist_ok=True)
    output_dir.mkdir(parents=True, exist_ok=True)

    with open(ls_json_path, 'r') as f:
        data = json.load(f)

    for task in data:
        image_path = Path(task['data']['image'])
        txt_filename = image_path.stem + '.txt'
        txt_path = output_dir / txt_filename

        annotations = task.get('annotations', [])
        if not annotations:
            continue
        annotation = annotations[0]  # Assuming one annotation per task

        image_width = None
        image_height = None
        # Collect image dimensions
        for res in annotation['result']:
            if res['type'] == 'rectanglelabels':
                image_width = res['original_width']
                image_height = res['original_height']
                break
        if image_width is None or image_height is None:
            continue  # Cannot process without image dimensions

        objects = []

        # Map rectangle IDs to their data
        rectangles = {}
        for result in annotation['result']:
            if result['type'] == 'rectanglelabels':
                bbox = result
                bbox_id = bbox['id']
                bbox_label = bbox['value']['rectanglelabels'][0]
                class_idx = class_map.get(bbox_label)
                if class_idx is None:
                    continue  # Skip unknown classes

                x = bbox['value']['x'] / 100
                y = bbox['value']['y'] / 100
                width = bbox['value']['width'] / 100
                height = bbox['value']['height'] / 100
                # Convert from top-left corner to center coordinates
                x_center = x + width / 2
                y_center = y + height / 2

                rectangles[bbox_id] = {
                    'class_idx': class_idx,
                    'x_center': x_center,
                    'y_center': y_center,
                    'width': width,
                    'height': height,
                    'keypoints': []
                }

        # Collect keypoints associated with each rectangle
        for kp_result in annotation['result']:
            if kp_result['type'] == 'keypointlabels':
                parent_id = kp_result.get('parentID')
                if parent_id in rectangles:
                    kp_x = kp_result['value']['x'] / 100
                    kp_y = kp_result['value']['y'] / 100
                    # Optional: Assign visibility (0: not labeled, 1: labeled but not visible, 2: visible)
                    visibility = 2  # Assuming keypoints are visible
                    if dims == 3:
                        rectangles[parent_id]['keypoints'].extend([kp_x, kp_y, visibility])
                    else:
                        rectangles[parent_id]['keypoints'].extend([kp_x, kp_y])

        # Prepare YOLO formatted lines
        lines = []
        for rect in rectangles.values():
            obj = [
                rect['class_idx'],
                rect['x_center'],
                rect['y_center'],
                rect['width'],
                rect['height']
            ] + rect['keypoints']
            line = ' '.join(map(str, obj))
            lines.append(line)

        # Write to YOLO format file
        with open(txt_path, 'w') as f:
            for line in lines:
                f.write(line + '\n')

In [24]:
output_dir = HOME/"annotations/keypoints/yolo"
output_dir.exists()

True

In [25]:
import shutil

In [26]:
if any(output_dir.iterdir()):
    shutil.rmtree(output_dir)
    output_dir.mkdir(parents=True)

In [27]:
class_map = {
    'clock': 0
    }  # Map your labels to class indices

In [28]:
convert_ls_to_yolo_kpt(KP_ANNOTATIONS, output_dir, class_map, dims=3)

In [29]:
len(list(output_dir.glob("*.txt")))

1200